## Drive Mount + Git Pull Push

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/Shareddrives/EECS\ 442\ -\ Project/eecs442-proj
!pip install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!git config --global user.email "wzahm@umich.edu"
#!git config --global user.name "Wyler Zahm"
#!git pull
!git add *
!git commit -m "COLAB COMMIT"
!git push
!git status
#!git pull
#!git add Executor.ipynb metrics.ipynb
#!git commit -m "Adding executor noteboook."

#!git push
#!git push

## Data Downloading

In [29]:
#!pip install -r requirements.txt
#%cd diode
#!wget http://diode-dataset.s3.amazonaws.com/val.tar.gz
#!tar -zxvf val.tar.gz
#%cd ..

#!wget https://download.pytorch.org/models/densenet169-b2777c0a.pth

!pip uninstall matplotlib
!pip install matplotlib

Uninstalling matplotlib-3.1.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.1.3
  Using cached https://files.pythonhosted.org/packages/ce/63/74c0b6184b6b169b121bb72458818ee60a7d7c436d7b1907bd5874188c55/matplotlib-3.4.1-cp37-cp37m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


## Model Exection

In [30]:
%load_ext autoreload
%autoreload 2
from model_pipeline import main as exec_model

exec_model()

[autoreload of matplotlib failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: module 'matplotlib.rcsetup' has no attribute '_validators'
]
[autoreload of matplotlib.cbook failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: connect() requires a code object with 0 free vars, not 4
]
/usr/lib/python3.7/importlib/__init__.py:169: MatplotlibDeprecationWarning: The module matplotlib.cbook.deprecation is considered internal and it will be made private in the future.
  _bootstrap._exec(spec, module)
[autoreload of matplotlib.rcsetup failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, relo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload




0it [00:00, ?it/s]


Training and Evaluating Basemodel with: 	LR = 0.0001	WD = 0


TypeError: ignored